# Write-Up Outline

1. Trial run with random model (~17)
2. Initial random forest test (untuned) (~14)
3. Initial neural network test

In [1]:
import numpy as np
import pandas as pd
import h2o

from sklearn.model_selection import train_test_split
from h2o.estimators import H2ORandomForestEstimator

h2o.init()

ModuleNotFoundError: No module named 'h2o'

In [182]:
def print_score(model, x, y):
    print('Score:        {:.4f}%'.format(model.score(x, y) * 100))
    print('Kaggle Score: {:.0f}'.format(np.sqrt(np.mean((model.predict(x) - y)**2))))   

In [183]:
df = pd.read_csv('data/stock_XY_train.csv')
df.head()

hdf = h2o.import_file('data/stock_XY_train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


# Custom Model

In [184]:
class CustomModel:
    
    def __init__(self):
        self.prediction = None
    
    def fit(self, x, y):
        pass
    
    def predict(self, x: np.ndarray):
        if self.prediction is None:
            self.prediction = np.random.choice([0, 1], size=len(x))
            
        return self.prediction
    
    def score(self, x, y):
        y_pred = self.predict(x)
        return sum([y_pred_point == y_point for y_pred_point, y_point in zip(y_pred, y)]) / len(y)
    

In [185]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(['Buy'], axis=1),
    df['Buy'],
    test_size=0.4,
    random_state=42,
)

custom_model = CustomModel()
custom_model.fit(x_train, y_train)
print_score(custom_model, x_test, y_test)

Score:        49.7779%
Kaggle Score: 1


# Random Forest

In [186]:
train, test = hdf.split_frame(ratios=[0.8])

x_columns = hdf.columns
x_columns.remove('Buy')
hrf = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)
hrf.train(
    x=x_columns,
    y='Buy',
    training_frame=train
)

hrf.model_performance(test_data=test)

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.19510438047668097
RMSE: 0.4417062151211832
MAE: 0.3814656147130432
RMSLE: 0.31072247371309464
Mean Residual Deviance: 0.19510438047668097


# Submission File Generation

In [187]:
dfTest = pd.read_csv('data/stock_X_test.csv').drop('Unnamed: 0', axis=1).rename({'Unnamed: 0.1': 'Unnamed: 0'}, axis='columns')
hdfTest = h2o.import_file('data/stock_X_test.csv')

x_columns = hdf.columns
x_columns.remove('Buy')
hrf = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)
hrf.train(
    x=x_columns,
    y='Buy',
    training_frame=hdf
)

hdfTest['Buy'] = hrf.predict(hdfTest)
hdfTest

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/home/josh/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Ticker' has levels not trained on: [ABCD, AET, AGIO, AHL, AIZP, AP, APLS, ARA, ATXI, AVD, BAK, BBRG, BBU, BFS, BNY, BQH, BRQS, BSD, BUD, CA, CAFD, CDLX, CGBD, CHI, CHU, CHUBA, CII, CLGN, CLGX, CLNC, COBZ, CORR, CRH, DCM, DFBHU, DMRC, DNB, EAST, EDF, EEP, EGN, EIO, EMJ, ENB, ERO, ETE, ETP, EVT, EYEN, FEDU, FFIC, FMY, FOX, FOXA, FPL, GBNK, GCVRZ, GER, GF, GIB, GPAQ, GXP, HCAC, HDP, HNI, HVBC, HYAC, HYACU, IAM, IFMK, IFN, ILG, IRTC, JKHY, JKS, JTPY, KALA, KANG, KFY, LCA, LCM, LEU, LGI, LKM, LOMA, MBI, MCA, MCI, MCR, MIE, MNLO, MOTS, MRBK, MSCI, MSD, MTGE, MUDS, MUDSU, MUI, NAC, NCSM, NEN, NIQ, NLST, NNC, NQP, NTR, NUO, NXP, NYH, OLP, OMAD, OPESU, OSS, PAGS, PCM, PERY, PHH, PKD, PNK, PRSS, PXD, QQQX, RWGE, SCPH, SCYX, SLDB, SLGL, SNDR, SOR, SPN, SPRO, STKL, SVA, TGEN, THG, TORC, TRMD, TRMT, TSRO, TTP, TVPT, VVPR, WDC, WEB, WHLRD, WPZ, WRLSU, XOG]
  warnings.warn(w)


C1,Unnamed: 0,Ticker,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,Income Tax Expense,Net Income - Non-Controlling int,Net Income - Discontinued ops,Net Income,Preferred Dividends,Net Income Com,EPS,EPS Diluted,Weighted Average Shs Out,Weighted Average Shs Out (Dil),Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin,Cash and cash equivalents,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,Long-term investments,Tax assets,Total non-current assets,Total assets,Payables,Short-term debt,Total current liabilities,Long-term debt,Total debt,Deferred revenue,Tax Liabilities,Deposit Liabilities,Total non-current liabilities,Total liabilities,Other comprehensive income,Retained earnings (deficit),Total shareholders equity,Investments,Net Debt,Other Assets,Other Liabilities,Depreciation & Amortization,Stock-based compensation,Operating Cash Flow,Capital Expenditure,Acquisitions and disposals,Investment purchases and sales,Investing Cash flow,Issuance (repayment) of debt,Issuance (buybacks) of shares,Dividend payments,Financing Cash Flow,Effect of forex changes on cash,Net cash flow / Change in cash,Free Cash Flow,Net Cash/Marketcap,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,ebitperRevenue,ebtperEBIT,niperEBT,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,nIperEBT,eBTperEBIT,eBITperRevenue,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,debtRatio,debtEquityRatio,longtermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatios,dividendpaidAndCapexCoverageRatios,dividendPayoutRatio,Revenue per Share,Net Income per Share,Operating Cash Flow per Share,Free Cash Flow per Share,Cash per Share,Book Value per Share,Tangible Book Value per Share,Shareholders Equity per Share,Interest Debt per Share,Market Cap,Enterprise Value,PE ratio,Price to Sales Ratio,POCF ratio,PFCF ratio,PB ratio,PTB ratio,EV to Sales,Enterprise Value over EBITDA,EV to Operating cash flow,EV to Free cash flow,Earnings Yield,Free Cash Flow Yield,Debt to Equity,Debt to Assets,Net Debt to EBITDA,Current ratio,Interest Coverage,Income Quality,Dividend Yield,Payout Ratio,SG&A to Revenue,R&D to Revenue,Intangibles to Total Assets,Capex to Operating Cash Flow,Capex to Revenue,Capex to Depreciation,Stock-based compensation to Revenue,Graham Number,ROIC,Return on Tangible Assets,Graham Net-Net,Working Capital,Tangible Asset Value,Net Current Asset Value,Invested Capital,Average Receivables,Average Payables,Average Inventory,Days Sales Outstanding,Days Payables Outstanding,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,10Y Revenue Growth (per Share),5Y Revenue Growth (per Share),3Y Revenue Growth (per Share),10Y Operating CF Grow

In [188]:
buy_list = list(hdfTest['Buy'])[0]

In [189]:
hdfTest.as_data_frame()[['Unnamed: 0', 'Buy']].to_csv('submission.csv', index=None, header=True)

In [190]:
dfTest[['Unnamed: 0', 'Buy']].to_csv('submission.csv', index=None, header=True)

KeyError: "['Buy'] not in index"